# **TP : Étude des Tests Statistiques**

### **Auteur : KHELID Lilya-Nada**

---

### **Objectifs :**

Dans ce notebook, nous allons explorer différents tests statistiques abordés en cours. Les thématiques principales sont :

1. **Visualisation des données.**  
2. **Tests d'adéquation à une loi.**  
3. **Comparaison de groupes.**  
4. **Influence de la taille des échantillons.**

---



In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from scipy.stats import (
    ttest_1samp, ttest_ind, mannwhitneyu, shapiro, kstest, wilcoxon,
    f_oneway, pearsonr, spearmanr, binom, fisher_exact,
    kruskal, friedmanchisquare, chisquare
)
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import lilliefors


### **Génération de Données : Loi Binomiale**

Considérons une distribution binomiale définie par :  
- $n$ : Nombre total d'essais  (n=trials)
- $p$ : Probabilité de succès  (p=probability)

La loi binomiale, notée $ \mathcal{B}(n, p) $, modélise le nombre de succès obtenus sur $n$ essais indépendants, chaque essai ayant une probabilité $p$ de succès.

Nous générons un échantillon de données suivant cette distribution pour les analyser et les visualiser.
  


In [2]:
trials = 15
probability = 1/3
size_of_array = 100
data_bin = np.random.binomial(trials,probability,size_of_array)


### **Génération de Données : Loi Normale**

Considérons une distribution normale définie par :  
- $ \mu $ : La moyenne (ou espérance) de la distribution  ($ \mu $ = nu)
- $ \sigma $ : L'écart type ($ \sigma $ = sigma)                              

La loi normale, notée $ \mathcal{N}(\mu, \sigma^2) $, modélise des données continues réparties symétriquement autour de leur moyenne $ \mu $, avec une dispersion déterminée par $ \sigma $. 

Nous générons un échantillon de données suivant cette distribution pour les analyser et les visualiser.


In [3]:
nu = 5
sigma = 2
size_of_array = 100
data_norm = np.random.normal(nu,sigma,size_of_array)

### **Visualisation des Données**

Pour mieux comprendre la répartition des données générées, nous les représentons graphiquement.  


In [4]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Distribution normale", "Distribution binomiale"])

fig.add_trace(go.Scatter(x=list(range(size_of_array)), y=data_norm, mode='markers', name="Normale"), row=1, col=1)
fig.add_hline(y=nu, row=1, col=1, line_dash="dash", line_color="red") #moyenne

fig.add_trace(go.Scatter(x=list(range(size_of_array)), y=data_bin, mode='markers', name="Binomiale"), row=1, col=2)
fig.add_hline(y=trials * probability, row=1, col=2, line_dash="dash", line_color="blue") #moyenne

fig.show()


In [5]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Distribution normale", "Distribution binomiale"])

fig.add_trace(go.Histogram(x=data_norm, nbinsx=20,marker_color="blue"), row=1, col=1,)
fig.add_trace(go.Histogram(x=data_bin, nbinsx=20,marker_color="red"), row=1, col=2)
fig.show()


### **Application des Tests Statistiques**

Afin d'analyser les propriétés des données générées, nous appliquons différents tests statistiques. Ces tests nous permettront de :  
- Vérifier des hypothèses concernant la distribution des données .  
- Comparer des groupes.  
- Évaluer des relations entre les variables.


### **Test de Normalité des Données**

#### **Hypothèses :**
- $H_0$ : Les données suivent une distribution normale.  
- $H_1$ : Les données ne suivent pas une distribution normale.

1. **Test de Shapiro-Wilk**  
   - Idéal pour des échantillons de petite à moyenne taille.
2. **Test de Kolmogorov-Smirnov**  
   - Permet de tester la correspondance entre la distribution empirique des données et une distribution, ici normale.
3. **Test de Lilliefors**  
   - Une variante du test de Kolmogorov-Smirnov adaptée lorsque les paramètres de la distribution normale ($\mu$ et $\sigma$) sont estimés à partir des données.

#### **Interprétation des résultats :**
- $p < 0.05$ : Nous rejetons $H_0$.  
- $p \geq 0.05$ : Nous ne rejetons pas $H_0$.


In [6]:
test_dict = {
            0 : 'shapiro',
            1 : 'kolmogorov smirnov',
            2 : 'lilliefors'
}


stat_shapiro, p_value_shapiro = shapiro(data_norm)

stat_ks, p_value_ks = kstest(data_norm, 'norm',args=(5, 2))

stat_lillie, p_value_lillie = lilliefors(data_norm)


for i,p_value in enumerate([p_value_shapiro,p_value_ks,p_value_lillie]):
    print(test_dict[i])
    print(p_value)
    if p_value >= 0.05:
        print('Les données suivent une distribution normale')
    else: 
        print("Les données ne suivent pas une distribution normale")
    print(' ')

shapiro
0.3648892939090729
Les données suivent une distribution normale
 
kolmogorov smirnov
0.6600012564007103
Les données suivent une distribution normale
 
lilliefors
0.05531113959937137
Les données suivent une distribution normale
 


>Le test de Lilliefors a échoué à valider la normalité, contrairement aux autres tests. Cela peut s'expliquer par :
>- Le test de Lilliefors est sensible aux petites déviations dues à l'aléa.  
>- Nous avons un trop petit échantillon


### **Tests de la Binomialité des Données**

#### **Hypothèses :**
- $H_0$ : Les données suivent une distribution binomiale.  
- $H_1$ : Les données ne suivent pas une distribution binomiale.

#### **Tests effectués :**

1. **Test de Kolmogorov-Smirnov (KS)**  
   - Objectif : Vérifier la correspondance entre la distribution empirique des données et une distribution binomiale.
   - Méthode : Compare la fonction de répartition empirique (ECDF) des données à la fonction de répartition théorique (CDF) de la loi binomiale.
   - Limitation : Le test KS est moins fiable pour des distributions discrètes comme la binomiale.

2. **Test du Chi-2**  
   - Objectif : Comparer les fréquences observées dans les données à celles attendues pour une loi binomiale.
   - Méthode : Calcule la somme des écarts normalisés entre les fréquences observées et théoriques.
   - Avantage : Plus adapté aux distributions discrètes que le test KS, surtout pour de grands échantillons.


In [7]:
n,p=15, 1/3
#--ks--
stat_ks, p_value_ks = kstest(data_bin, 'binom', args=(n, p))
print("P-value du test KS :", p_value_ks)
if p_value_ks >= 0.05:
    print("Les données suivent une distribution binomiale (KS Test)")
else: 
    print("Les données ne suivent pas une distribution binomiale (KS Test)")

#--chi2--
observed, _ = np.histogram(data_bin, bins=range(n + 2)) #pour compter les occurences

expected = size_of_array * np.array([binom.pmf(k, n, p) for k in range(n + 1)])

stat_chi2, p_value_chi2 = chisquare(f_obs=observed, f_exp=expected)
print("P-value du test Chi-2 :", p_value_chi2)
if p_value_chi2 >= 0.05:
    print("Les données suivent une distribution binomiale (Chi-2 Test)")
else: 
    print("Les données ne suivent pas une distribution binomiale (Chi-2 Test)")


P-value du test KS : 2.241550081622303e-06
Les données ne suivent pas une distribution binomiale (KS Test)
P-value du test Chi-2 : 0.3122966131557384
Les données suivent une distribution binomiale (Chi-2 Test)


### **Comparaison de Groupes**

Pour comparer les groupes et évaluer s'ils présentent des différences significatives, nous appliquons des tests statistiques adaptés. Ces tests permettent de comparer les moyennes, les distributions ou d'autres métriques entre plusieurs groupes.  

#### **Hypothèses :**
- $H_0$ : Les groupes sont identiques selon la métrique étudiée.  
- $H_1$ : Les groupes présentent des différences significatives.
 
1. **Test t de Student** : Comparaison des moyennes pour deux groupes. 
En cas de petit échantillon ou de non-respect des hypothèses, des alternatives comme le test de Welch (ou Aspin-Welch) sont recommandées.
2. **Test de Mann-Whitney U** : Comparaison non paramétrique pour deux groupes.   


#### **Interprétation des résultats :**
- $p < 0.05$ : Nous rejetons $H_0$.  
- $p \geq 0.05$ : Nous ne rejetons pas $H_0$.



In [8]:
print("=== Test t pour une moyenne spécifique ===")
t_stat, p_value = ttest_1samp(data_norm, 5)
print(f"Statistique t : {t_stat:.4f}")
print(f"Valeur p : {p_value:.4f}")
if p_value >= 0.05:
    print("La moyenne des données correspond à la valeur théorique attribuée (5).")
else:
    print("La moyenne des données ne correspond pas à la valeur théorique attribuée (5).")
print(" ")

print("=== Test de Mann-Whitney ===")
stat, p_value = mannwhitneyu(data_norm, data_bin, alternative='two-sided')
print(f"Statistique U : {stat:.4f}")
print(f"Valeur p : {p_value:.4f}")
if p_value < 0.05:
    print("Il y a une différence significative entre les données normal et binomiale..")
else:
    print("Il n'y a pas de différence significative entre les données normal et binomiale.")

=== Test t pour une moyenne spécifique ===
Statistique t : -0.4906
Valeur p : 0.6248
La moyenne des données correspond à la valeur théorique attribuée (5).
 
=== Test de Mann-Whitney ===
Statistique U : 4789.0000
Valeur p : 0.6064
Il n'y a pas de différence significative entre les données normal et binomiale.


### Comparaison de Groupes : ANOVA, Friedman, Kruskal-Wallis et Wilcoxon

Dans cette section, nous explorons quatre méthodes de comparaison de groupes : **ANOVA**, **Friedman**, **Kruskal-Wallis**, et **Wilcoxon**. Ces tests sont utilisés selon les types de données et les hypothèses associées.

---

#### 1. ANOVA (Analyse de la Variance)

**Objectif** : Comparer les moyennes de plusieurs groupes (> 2) pour déterminer si au moins une moyenne est significativement différente des autres.

**Hypothèses** :
- **$H_0$** : Toutes les moyennes des groupes sont égales.
- **$H_1$** : Au moins une moyenne est différente.

**Conditions d'application** :
1. Les données doivent suivre une loi normale dans chaque groupe.
2. Les variances des groupes doivent être homogènes.
3. Les échantillons doivent être indépendants.

**Exemple d'utilisation** : Comparer les scores moyens d’étudiants ayant suivi trois méthodes d’enseignement différentes.

---

#### 2. Test de Friedman

**Objectif** : Comparer les médianes de plusieurs groupes appariés (> 2) pour déterminer s’il existe des différences significatives.

**Hypothèses** :
- **$H_0$** : Les distributions des groupes sont identiques.
- **$H_1$** : Au moins un groupe a une distribution différente.

**Conditions d'application** :
1. Les données doivent être ordinales ou continues.
2. Les groupes doivent être appariés (par exemple, mesures répétées sur les mêmes sujets).
3. Pas besoin de normalité ni d’homogénéité des variances.

**Exemple d'utilisation** : Comparer les temps de réaction de participants à trois moments différents (matin, midi, soir).

---

#### 3. Test de Kruskal-Wallis

**Objectif** : Comparer les distributions (ou médianes) de plusieurs groupes indépendants (> 2).

**Hypothèses** :
- **$H_0$** : Les distributions des groupes sont identiques.
- **$H_1$** : Au moins un groupe a une distribution différente.

**Conditions d'application** :
1. Les données doivent être ordinales ou continues.
2. Les groupes doivent être indépendants.
3. Pas besoin de normalité ni d’homogénéité des variances.

**Exemple d'utilisation** : Comparer les scores de satisfaction de clients entre trois magasins différents.

---

#### 4. Test de Wilcoxon (rangs signés)

**Objectif** : Comparer les distributions ou médianes de deux groupes appariés.

**Hypothèses** :
- **$H_0$** : Les médianes des différences entre les paires sont égales à 0.
- **$H_1$** : Les médianes des différences entre les paires sont différentes de 0.

**Conditions d'application** :
1. Les données doivent être ordinales ou continues.
2. Les groupes doivent être appariés.
3. Pas besoin de normalité.

**Exemple d'utilisation** : Comparer le poids des patients avant et après un traitement.

---

#### 5. Tableau comparatif des tests

| Test               | Type de données     | Groupes indépendants ? | Appariés ? | Normalité requise ? | Compare quoi ?       |
|---------------------|---------------------|-------------------------|------------|---------------------|----------------------|
| ANOVA              | Continues          | Oui                     | Non        | Oui                 | Moyennes            |
| Friedman           | Ordinales/Continues| Non                     | Oui        | Non                 | Médianes            |
| Kruskal-Wallis     | Ordinales/Continues| Oui                     | Non        | Non                 | Médianes            |
| Wilcoxon           | Ordinales/Continues| Non                     | Oui        | Non                 | Médianes ou distrib. |



# Évaluation de l'impact de la taille des échantillons sur les tests d'adéquation

Cette analyse vise à évaluer l'impact de la taille des échantillons sur les tests d'adéquation à une loi pour :

- **Normale**
- **Binomiale**

L'évaluation sera réalisée sur **100 itérations** pour chaque loi et chaque taille d'échantillon.


In [9]:
p_values_shapiro, p_values_ks, p_values_lillie = [], [], []
nu, sigma = 0, 1

sample_sizes = range(5, 1005, 10) #100 itérations
for size in sample_sizes:
    data = np.random.normal(nu, sigma, size)

    p_values_shapiro.append(shapiro(data)[1])
    p_values_ks.append(kstest(data, 'norm', args=(nu, sigma))[1])
    p_values_lillie.append(lilliefors(data)[1])


fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=["Shapiro-Wilk", "Kolmogorov-Smirnov", "Lilliefors"])


tests = [("Shapiro-Wilk", p_values_shapiro), 
         ("Kolmogorov-Smirnov", p_values_ks), 
         ("Lilliefors", p_values_lillie)]

for i, (name, p_values) in enumerate(tests, start=1):
    fig.add_trace(
        go.Scatter(x=list(sample_sizes), y=p_values, mode='markers', name=name),
        row=i, col=1
    )
    fig.add_hline(y=0.05, line_dash="dash", line_color="red", row=i, col=1)


fig.update_layout(title="P-values des tests de normalité", 
                  xaxis_title="Taille de l'échantillon généré", 
                  yaxis_title="P-value", 
                  height=800, 
                  template="plotly_white")
fig.show()

>En zoomant sur les tailles d'échantillons faibles, on note que Lilliefors ne fait pas d'erreur. Shapiro est également très performant sur les échantillons de petite taille, tandis que KS se trompe parfois.

In [10]:
# Initialisation des listes pour stocker les p-values
p_values_ks, p_values_chi2 = [], []

# Paramètres de la loi binomiale
n, p = 10, 0.5  # n essais, probabilité de succès p
sample_sizes = range(10, 1010, 10)

# Simulation et tests
for size in sample_sizes:
    data = np.random.binomial(n, p, size)

    #--ks--
    stat_ks, p_value_ks = kstest(data, 'binom', args=(n, p))
    p_values_ks.append(p_value_ks)

    #--chi2--
    observed, _ = np.histogram(data, bins=range(n + 2))  # Distribution observée
    expected = size * np.array([binom.pmf(k, n, p) for k in range(n + 1)])  # Distribution théorique
    stat_chi2, p_value_chi2 = chisquare(f_obs=observed, f_exp=expected)
    p_values_chi2.append(p_value_chi2)


fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=["Kolmogorov-Smirnov", "Chi-2"])


fig.add_trace(
    go.Scatter(x=list(sample_sizes), y=p_values_ks, mode='markers', name="KS Test"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=list(sample_sizes), y=p_values_chi2, mode='markers', name="Chi-2 Test"),
    row=2, col=1
)

# Ajout des lignes alpha
for i in range(1, 3):
    fig.add_hline(y=0.05, line_dash="dash", line_color="red", row=i, col=1)

# Mise en forme
fig.update_layout(title="P-values des tests d'adéquation pour une loi binomiale",
                  xaxis_title="Taille de l'échantillon",
                  yaxis_title="P-value",
                  height=600,
                  template="plotly_white")

# Affichage
fig.show()


In [11]:
p_values_ks, p_values_chi2 = [], []


n, p = 10, 0.5  # n essais, probabilité de succès p

sample_sizes = range(10, 1010, 10)
for size in sample_sizes:
    data = np.random.binomial(n, p, size)

    #--ks--
    stat_ks, p_value_ks = kstest(data, 'binom', args=(n, p))
    p_values_ks.append(p_value_ks)

    #--chi2--
    observed, _ = np.histogram(data, bins=range(n + 2))  # Distribution observée
    expected = size * np.array([binom.pmf(k, n, p) for k in range(n + 1)])  # Distribution théorique
    stat_chi2, p_value_chi2 = chisquare(f_obs=observed, f_exp=expected)
    p_values_chi2.append(p_value_chi2)


fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=["Kolmogorov-Smirnov", "Chi-2"])


fig.add_trace(
    go.Scatter(x=list(sample_sizes), y=p_values_ks, mode='markers', name="KS Test"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=list(sample_sizes), y=p_values_chi2, mode='markers', name="Chi-2 Test"),
    row=2, col=1
)

# Ajout des lignes alpha = 0.05
for i in range(1, 3):
    fig.add_hline(y=0.05, line_dash="dash", line_color="red", row=i, col=1)


fig.update_layout(title="P-values des tests d'adéquation pour une loi binomiale",
                  xaxis_title="Taille de l'échantillon",
                  yaxis_title="P-value",
                  height=600,
                  template="plotly_white")


fig.show()


### **Analyse de l'adéquation des tests pour une loi binomiale**

1. **Limite du test de Kolmogorov-Smirnov (KS)**  
   - Le test KS n'est **pas adapté** pour une distribution **discrète** comme la loi binomiale.  
   - Cela s'explique par le fait que KS compare des fonctions de répartition (CDF), et pour une distribution discrète, les sauts dans la CDF rendent la comparaison imprécise.

2. **Efficacité du test du Chi-2**  
   - En revanche, le **test du Chi-2** est très performant pour détecter la conformité à une loi binomiale.  
   - Il compare directement les **fréquences observées** avec les **fréquences attendues**, ce qui est particulièrement adapté aux distributions discrètes.


# **Conclusion**

Dans ce notebook, nous avons exploré plusieurs thématiques clés en statistique, notamment :

1. **Tests d'adéquation de loi** :  
   - Analyse des méthodes permettant de vérifier si un jeu de données suit une distribution théorique.
   - Étude des tests compatibles avec les lois **normale** et **binomiale**, tels que :
     - **Kolmogorov-Smirnov (KS)** : Adapté pour des distributions continues, mais moins performant pour des distributions discrètes comme la loi binomiale.
     - **Test du Chi-2** : Particulièrement efficace pour les distributions discrètes comme la loi binomiale.

2. **Tests de comparaison de groupes** :  
   - Tests statistiques pour comparer des échantillons provenant de groupes différents.
   - Mise en lumière de l'importance des hypothèses sous-jacentes (normalité, égalité des variances, etc.).

3. **Impact de la taille des échantillons** :  
   - Étude de l'influence de la taille des échantillons sur la robustesse et la fiabilité des tests d'adéquation de loi.
   - Illustration des performances des tests sur des tailles d'échantillons faibles et élevées pour les lois normales et binomiales.

---

### **Points clés à retenir :**
- Le choix du test statistique dépend de la nature des données (continue ou discrète) et de la distribution théorique étudiée.
- Les **tailles d'échantillons faibles** peuvent entraîner une perte de puissance pour certains tests, nécessitant des ajustements ou des alternatives robustes.
- La combinaison de plusieurs tests (par exemple, KS et Chi-2) permet une analyse plus complète et fiable.




In [12]:
#jupyter nbconvert --to html --execute Tests_statistiques_KHELID.ipynb && open Tests_statistiques_KHELID.html